In [ ]:
#Mounting the dataset to google drive

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Installing the necessary libraries

!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken

In [ ]:
import os

# Prompt the user for their OpenAI API key
#api_key = input("Please enter your OpenAI API key: ")

api_key="sk-proj-JGSydOhnADiszdMjJornT3BlbkFJCQXUHBBzSqrDSCHookxZ"

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")


OPENAI_API_KEY has been set!


In [ ]:
llm_model = "gpt-3.5-turbo"

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

In [ ]:
txt_file_path = '/content/drive/MyDrive/project/llm/Travel_AI.csv'
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [ ]:
data

[Document(page_content='\ufeffplace,time,about_trip\n"Orlando, Florida",7,"Day 1: Arrival and check-in at a family-friendly resort. Visit Disney Springs for dinner and shopping.\nDay 2: Full day at Magic Kingdom Park, with a focus on classic attractions suitable for all ages.\nDay 3: Explore EPCOT, enjoying the Future World and the World Showcase.\nDay 4: Spend the day at Universal Studios Florida, with rides and shows from favorite movies.\nDay 5: Visit Universal’s Islands of Adventure, home to the Wizarding World of Harry Potter.\nDay 6: Relax at the resort’s pool and enjoy kid-friendly activities offered by the hotel. \nBudget: Accommodation: $150 - $300 per night for a family-friendly resort.\nTheme Park Tickets: $100 - $150 per person per park.\nDining: $30 - $50 per person per day.\nMiscellaneous (souvenirs, rentals, etc.): $20 - $40 per day.\nDay 7: Departure day. Have a character breakfast at the resort before heading home."\nItaly,14,"Days 1-4: Rome - Colosseum, Roman Forum, V

In [ ]:
# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Prompt 1 : Generating a  7-day itinerary for a family vacation to Orlando Florida.

In [ ]:
query = "Provide a full 7-day itinerary for a family vacation to Orlando, Florida"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'Day 1: Arrival and check-in at a family-friendly resort. Visit Disney Springs for dinner and shopping.\n\nDay 2: Full day at Magic Kingdom Park, with a focus on classic attractions suitable for all ages.\n\nDay 3: Explore EPCOT, enjoying the Future World and the World Showcase.\n\nDay 4: Spend the day at Universal Studios Florida, with rides and shows from favorite movies.\n\nDay 5: Visit Universal’s Islands of Adventure, home to the Wizarding World of Harry Potter.\n\nDay 6: Relax at the resort’s pool and enjoy kid-friendly activities offered by the hotel. \n\nDay 7: Departure day. Have a character breakfast at the resort before heading home. \n\nBudget overview: \n- Accommodation: $150 - $300 per night for a family-friendly resort.\n- Theme Park Tickets: $100 - $150 per person per park.\n- Dining: $30 - $50 per person per day.\n- Miscellaneous (souvenirs, rentals, etc.): $20 - $40 per day.'

Prompt 2 : Generating a two week itinerary for a historical tour of Italy.

In [ ]:
query = "Generate a two-week itinerary for a cultural and historical tour of Italy."
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"Sure, here's a proposed 14-day itinerary for a cultural and historical tour of Italy:\n\nDays 1-4: Rome \n- Visit the historic Colosseum, marvel at the remnants of the Roman Forum, and explore the vast Vatican Museums.\n- Enjoy dining in local trattorias and try traditional Roman dishes like carbonara and supplì.\n- Budget: Accommodation: $100 - $200 per night for mid-range hotels. Museum and Historical Site Tickets: $20 - $30 per site. Dining: $40 - $70 per person per day. Transportation (trains, local transit): $30 - $50 per day.\n\nDays 5-6: Travel to Naples \n- Visit the historic city center, a UNESCO World Heritage site, explore the Royal Palace and the Castel dell'Ovo.\n- Don't miss the opportunity to taste the authentic Neapolitan pizza.\n\nDays 7-8: Pompeii and Amalfi Coast\n- Spend a day in Pompeii, exploring the ancient ruins of the city that was buried by the eruption of Mt. Vesuvius in 79 A.D.\n- Next day, take a scenic drive along the Amalfi Coast, visiting the towns of P

Prompt 3 : Generating an itinerary for a safari in Kenya.

In [ ]:
query = "Create a detailed 10-day itinerary for an adventure and wildlife safari in Kenya."
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'Days 1-2: Nairobi - Start your adventure in the capital city of Nairobi. Visit the David Sheldrick Wildlife Trust to see the orphaned elephants and the Giraffe Centre to feed giraffes.\n\nDays 3-6: Maasai Mara - Transfer to Maasai Mara, one of the most famous wildlife conservation areas in the world. Enjoy safari drives where you can spot the big five (lion, leopard, rhinoceros, elephant, and Cape buffalo). Visit a Maasai village to learn about their culture and traditions. Stay in eco-lodges for a unique experience close to nature.\n\nDays 7-10: Tsavo - Continue your safari adventure in Tsavo, known for its diverse landscapes and large herds of elephants. Enjoy more safari drives, visit the Mzima Springs, and relax in luxury tented camps.\n\nBudget: \nAccommodation: Expect to pay between $150 - $500 per night for lodges or eco-camps.\nSafari Tours: These can range from $200 - $400 per day.\nDining: Most safari packages include meals.\nMiscellaneous expenses like tips and souvenirs ca

Prompt 4 :  Generating a one week getaway to Maldives

In [ ]:
query = "Describe a one-week romantic getaway to the Maldives."
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"Day 1: Arrive in Malé, the capital of the Maldives. Spend the day exploring the city's colorful buildings, bustling markets, and historical sites. Enjoy a traditional Maldivian dinner at a local restaurant.\n\nDay 2: Transfer to Maafushi Island, known for its stunning beaches and azure waters. Relax on the beach, swim in the crystal clear ocean, and soak in the sun. In the evening, enjoy a romantic dinner under the stars with Maldivian cuisine.\n\nDay 3: Engage in exciting water activities. Go snorkeling to explore the vibrant coral reefs and diverse marine life. Try out jet skiing for some adrenaline rush. \n\nDay 4: Visit a luxury private island resort. Spend the day pampering yourselves with spa treatments and unwind in the serene ambiance of the resort. Enjoy a fine dining experience at the resort's restaurant.\n\nDay 5: Travel to Fulidhoo Island to immerse yourselves in local culture. Explore the island, interact with the locals and understand their way of life. In the evening, e

Prompt 5 : Generating a comprehensive 10 day itinerary to explore Japan.

In [ ]:
query = "Create a comprehensive 10-day itinerary for exploring Japan."
result = conversation_chain({"question": query})
answer = result["answer"]
answer

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


"Day 1-3: Tokyo, Japan\n\nUpon arrival in Tokyo, settle into your accommodation. Start your exploration by visiting Ueno Park, famous for its museums, zoo, and cherry blossom trees. Spend the next day in Shinjuku Gyoen, another prime location for cherry blossom viewing. End your day with a visit to the bustling shopping district of Shibuya.\n\nDay 4-6: Kyoto, Japan\n\nTransfer to Kyoto and check into your accommodation. Start your Kyoto journey by attending a traditional tea ceremony. Spend the next day exploring the Philosopher’s Path, a scenic walk along a canal lined with hundreds of cherry trees. In the evenings, relax in a ryokan, a traditional Japanese inn.\n\nDay 7-8: Hiroshima, Japan\n\nTravel to Hiroshima and visit the Peace Memorial Park, a memorial dedicated to the victims of the atomic bomb. Spend your second day in Hiroshima visiting Miyajima Island, famous for its Itsukushima Shrine and more cherry blossom viewing spots.\n\nDay 9: Osaka, Japan\n\nTransfer to Osaka and spe